# Ejemplo archivos
1. Creamos la SparkSession.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import string

spark = SparkSession.builder\
  .appName("arquivo-example-1")\
  .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
  .getOrCreate()


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d8da22e3-ec97-44c3-8a74-2f2c2b632f14;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 187ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0 

2. Obtenemos el esquema e indicando el origen de datos para el procesamiento en streaming, en este caso una carpeta con archivos *json*.

In [2]:
path = "/data/flight-data/json"
static = spark.read.json(path)
dataSchema = static.schema
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
  .json(path)
streaming.printSchema()

[Stage 0:>                                                          (0 + 2) / 2]

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



3. Obtenemos el DataFrame de salida transformando los datos iniciales. En este caso agrupamos por *DEST_COUNTRY_NAME* y contamos.

In [4]:
counts = streaming.groupBy("DEST_COUNTRY_NAME").count()

4. Iniciamos el procesamiento en streaming con salida a memoria.

In [5]:
query = counts.writeStream.queryName("counts").format("memory").outputMode("complete").start()

[Stage 2:=============================>                         (109 + 2) / 200]

5. Mostramos los datos:

In [6]:
from time import sleep
for x in range(5):
    spark.sql("SELECT * FROM counts").show()
    sleep(1)

[Stage 2:================================>                      (117 + 2) / 200]

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
+-----------------+-----+



[Stage 2:========================================>              (147 + 2) / 200]

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
+-----------------+-----+



[Stage 2:=================================================>     (181 + 2) / 200]

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
+-----------------+-----+



+--------------------+-----+
|   DEST_COUNTRY_NAME|count|
+--------------------+-----+
|            Paraguay|    1|
|            Anguilla|    1|
|              Russia|    1|
|             Senegal|    1|
|              Sweden|    1|
|            Kiribati|    1|
|              Guyana|    1|
|         Philippines|    1|
|            Malaysia|    1|
|           Singapore|    1|
|                Fiji|    1|
|              Turkey|    1|
|             Germany|    1|
|         Afghanistan|    1|
|              Jordan|    1|
|               Palau|    1|
|              France|    1|
|Turks and Caicos ...|    1|
|              Greece|    1|
|            Dominica|    1|
+--------------------+-----+
only showing top 20 rows



+--------------------+-----+
|   DEST_COUNTRY_NAME|count|
+--------------------+-----+
|            Paraguay|    1|
|            Anguilla|    1|
|              Russia|    1|
|             Senegal|    1|
|              Sweden|    1|
|            Kiribati|    1|
|              Guyana|    1|
|         Philippines|    1|
|            Malaysia|    1|
|           Singapore|    1|
|                Fiji|    1|
|              Turkey|    1|
|             Germany|    1|
|         Afghanistan|    1|
|              Jordan|    1|
|               Palau|    1|
|              France|    1|
|Turks and Caicos ...|    1|
|              Greece|    1|
|            Dominica|    1|
+--------------------+-----+
only showing top 20 rows

